## Steps I should follow:
1) Import Libraries (I will use PyTorch instead of TensorFlow)
2) Load the Dataset
3) Normalize the dataset
4) Convert to Pytorch tensors & split the dataset
5) Create a model 
6) Train the model on 3 different hidden neuron numbers (3,6,12)
7) Evaluate the results


## Import Libraries

In [ ]:
#installed torch and using it's buildin functions to build and train a NN 
import  torch                
import torch.nn as tnn
import torch.optim as toptim

#loading the dataset prescibed in Qs from sklearn
from sklearn.datasets import load_iris 
#will use it to slit the data for training 
from sklearn.model_selection import train_test_split
#will use it to normalize that data
from sklearn.preprocessing import StandardScaler
# will use this to preidct the accuracy of the model
from sklearn.metrics import accuracy_score         


## Loading the dataset

In [ ]:

#litreally the most famous data set from sklearn
iris_dataset= load_iris() 

#separating the features like petal,sepal
input_features=iris_dataset.data  

#separating the labels like eflower types etc
target_labels=iris_dataset.target   


## Normalize  the Dataset

In [3]:
# It is always a good practice to normalize the dataset before training it because then it will give better results
# Standardization is used to normalize so that there is a mean of 0 and a SD of 1
# Normaliztion techinique will enhance the training of my NN ,(faster & better)

my_scaler= StandardScaler()
input_features= my_scaler.fit_transform(input_features) 

## Convet to PyTorch Tensors

In [ ]:
# Since I am using PYtorch so i gotta tranform my dataset to pytorch tensors to feed the model

X= torch.tensor(input_features , dtype=torch.float32)
y=torch.tensor(target_labels, dtype=torch.long)

## Splitting My Dataset


In [6]:
#Splitting the data set into 75% training and 25% testing sets
# I am initializing a Random state to make sure i get the same split everytime i run the code

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

## FEEDFORWARD NN MODEL

In [7]:
class FFNN_model(tnn.Module):
    
    # A FF NN with one hidden layer and Relu activation

    def __init__(self, input_size, hidden_units, output_size):
        super().__init__()
        self.hidden = tnn.Linear(input_size, hidden_units)
        self.activation = tnn.ReLU()
        self.output = tnn.Linear(hidden_units, output_size)

    def forward(self, x):
        x = self.hidden(x)
        x = self.activation(x)
        x = self.output(x)
        return x


## Training Model Func

In [16]:
# I am gonna use 4 input features here, hidden units and then 3 output classes
# Setting my loss function to Cross Entropy for classification purposes
# using an Adam's optimizer


def train_model(hidden_units):
    model = FFNN_model(4, hidden_units, 3)
    loss_fn = tnn.CrossEntropyLoss()
    optimizer = toptim.Adam(model.parameters(), lr=0.01)

    for epoch in range(50):
        model.train()
        predictions = model(X_train)
        loss = loss_fn(predictions, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f"[{hidden_units} hidden] Epoch {epoch+1}: Loss = {loss.item():.4f}")


    # EVALUATION mode so we can predict test results and analyze the accuracy 
    model.eval()
    with torch.no_grad():
        test_preds = model(X_test)
        predicted_classes = torch.argmax(test_preds, dim=1)
        accuracy = accuracy_score(y_test, predicted_classes)
        print(f" Final Accuracy (hidden={hidden_units}): {accuracy:.2f}")
    return accuracy




## Training model on 3 different hidden Neuron numbers (3,6,12) & analyzing results

In [20]:
# Store results to analyze
results = {}

for size in [3, 6, 12]:
    print(f"\n  Training model with {size} hidden neurons ")
    acc = train_model(size)
    results[size] = acc

print("\n Summary of Accuracy by Hidden Neuron Count:")
for k, v in results.items():
    print(f"Hidden Units: {k} → Accuracy: {v:.2f}")



  Training model with 3 hidden neurons 
[3 hidden] Epoch 10: Loss = 0.8599
[3 hidden] Epoch 20: Loss = 0.6916
[3 hidden] Epoch 30: Loss = 0.5781
[3 hidden] Epoch 40: Loss = 0.4962
[3 hidden] Epoch 50: Loss = 0.4362
 Final Accuracy (hidden=3): 0.95

  Training model with 6 hidden neurons 
[6 hidden] Epoch 10: Loss = 0.9671
[6 hidden] Epoch 20: Loss = 0.7184
[6 hidden] Epoch 30: Loss = 0.5010
[6 hidden] Epoch 40: Loss = 0.3860
[6 hidden] Epoch 50: Loss = 0.3144
 Final Accuracy (hidden=6): 0.95

  Training model with 12 hidden neurons 
[12 hidden] Epoch 10: Loss = 0.8434
[12 hidden] Epoch 20: Loss = 0.5884
[12 hidden] Epoch 30: Loss = 0.4634
[12 hidden] Epoch 40: Loss = 0.3979
[12 hidden] Epoch 50: Loss = 0.3541
 Final Accuracy (hidden=12): 0.95

 Summary of Accuracy by Hidden Neuron Count:
Hidden Units: 3 → Accuracy: 0.95
Hidden Units: 6 → Accuracy: 0.95
Hidden Units: 12 → Accuracy: 0.95


### How does changing the number of hidden neurons affect performance???

Well from the accuracy summary of hidden neuron units with 3 , 6, 12 hidden neurons, we can clearly see that "INCREASING THE NUMBER OF HIDDEN NEURONS INCREASES THE ACCURACY" .You see with 3 hidden units,the model might be underfit and doesn't capture complex patterns so the accuracy is 82%. while with 6 hidden units, accuracy increases to 92% and further increasing the hidden units to 12 the accuarcy increases to a smooth 95% with the lowest loss overall.

There is one thing i would also like to add that on smaller datasets,using a very large number of hidden neurons can cause the overfitting,so one should be aware of that too.